## Introduction to choice-learn's modelling

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
from pathlib import Path

sys.path.append("../")

import numpy as np
import pandas as pd

### ChoiceModel - Getting Started

choice-learn package offers a high level API to conceive and estimate discrete choice models.

We begin this tutorial with the estimation of a Conditional Logit Model on the ModeCanada dataset.

First, we download our data as a ChoiceDataset. See the data management tutorial first if needed.

In [ ]:
from choice_learn.datasets import load_modecanada

transport_df = load_modecanada(as_frame=True)

# Following torch-choice guide:
transport_df = transport_df.loc[transport_df.noalt == 4]

items = ["air", "bus", "car", "train"]

transport_df["oh_air"] = transport_df.apply(lambda row: 1. if row.alt == items[0] else 0., axis=1)
transport_df["oh_bus"] = transport_df.apply(lambda row: 1. if row.alt == items[1] else 0., axis=1)
transport_df["oh_car"] = transport_df.apply(lambda row: 1. if row.alt == items[2] else 0., axis=1)
transport_df["oh_train"] = transport_df.apply(lambda row: 1. if row.alt == items[3] else 0., axis=1)

transport_df.income = transport_df.income.astype("float32")

In [ ]:
from choice_learn.data import ChoiceDataset
dataset = ChoiceDataset.from_single_df(df=transport_df,
                                       fixed_items_features_columns=["oh_air", "oh_bus", "oh_car", "oh_train"],
                                       contexts_features_columns=["income"],
                                       contexts_items_features_columns=["cost", "freq", "ovt", "ivt"],
                                       items_id_column="alt",
                                       contexts_id_column="case",
                                       choices_column="choice",
                                       choice_mode="one_zero")

Now, we can import the needed modules:

In [ ]:
from choice_learn.models import ConditionalMNL

The conditional MNL \ref{Train} specifies a linear utility for each item i during the session s with regards to the features:
$$
U(i, s) = \sum_{features} a(i, s) * feat(i, s)
$$

We will use a ModelSpecification object to define our model with regards to our ChoiceDataset.
For each feature in the choice dataset we can specify how it must be specified in the utility.

Let's re-use a common example from \ref{} on the ModeCanda dataset:
$$
U(i, s) = \beta^{inter}_i + \beta^{price} \cdot price(i, s) + \beta^{freq} \cdot freq(i, s) + \beta^{ovt} \cdot ovt(i, s) + \beta^{income}_i \cdot income(s) + \beta^{ivt}_i \cdot ivt(i, t) + \epsilon(i, t)
$$

Note that we want to estimate:

- one $\beta^{price}$, $\beta^{freq}$ and $\beta^{ovt}$ coefficient. They are shared by all items.
- one $\beta^{ivt}$ coefficient for **each** item.
- one $\beta^{inter}$ and $\beta^{income}$ coefficient for **each** item, with **additional constraint** to be 0 for the first item (air).

We will use a ModelSpecification object to create the right utility function.
We need to specify for each weight:
- a unique name
- the name of the feature it goes with:
    - it must match the feature name in the ChoiceDataset
    - "intercept" is the standardized name used for intercept, pay attention not to override it
- items_indexes: the items concerned, as indexed in the ChoiceDataset

In [ ]:
# Initialization of the model
model = ConditionalMNL(optimizer="lbfgs")

# Creation of the different weights:


# add_coefficients adds one coefficient for each specified item_index
# intercept, and income are added for each item except the first one that needs to be zeroed
model.add_coefficients(coefficient_name="beta_inter", feature_name="intercept", items_indexes=[1, 2, 3])
model.add_coefficients(coefficient_name="beta_income", feature_name="income", items_indexes=[1, 2, 3])

# ivt is added for each item:
model.add_coefficients(coefficient_name="beta_ivt", feature_name="ivt", items_indexes=[0, 1, 2, 3])

# shared_coefficient add one coefficient that is used for all items specified in the items_indexes:
# Here, cost, freq and ovt coefficients are shared between all items
model.add_shared_coefficient(coefficient_name="beta_cost", feature_name="cost", items_indexes=[0, 1, 2, 3])
model.add_shared_coefficient(coefficient_name="beta_freq", feature_name="freq", items_indexes=[0, 1, 2, 3])
model.add_shared_coefficient(coefficient_name="beta_ovt", feature_name="ovt", items_indexes=[0, 1, 2, 3])

Now, we can instantiate our ConditionalMNL from the specification. We use LBFGS as the estimation method.

In order to estimate the the coefficients values, use the .fit method with the ChoiceDataset:

In [ ]:
history = model.fit(dataset, n_epochs=1000)

It is possible to see the estimated coefficients with the .weights argument:

In [ ]:
model.weights

The negative loglikelihood can be estimated using .evaluate():

In [ ]:
print("The average neg-loglikelihood is:", model.evaluate(dataset).numpy())
print("The total neg-loglikelihood is:", model.evaluate(dataset).numpy()*len(dataset))

A faster specification can be done using a dictionnary. It follows torch-choice \ref{} method to create conditional logit models.
The parameters dict needs to be as follows:
- The key is the feature name
- The value is the mode. Currently three modes are available:
    - constant: the learned coefficient is shared by all items
    - item: one coefficient by item is estimated, the value for the item at index 0 is set to 0
    - item-full: one coefficient by item is estimated

In order to create the same model for the ModeCanada dataset, it looks as follows:

In [ ]:
# Instantiation of the parameters dictionnary
params = {"income": "item",
 "cost": "constant", 
 "freq": "constant",
 "ovt": "constant", 
 "ivt": "item-full",
 "intercept": "item"}

# Instantiation of the model
cmnl = ConditionalMNL(parameters=params, optimizer="lbfgs")

In [ ]:
history = cmnl.fit(dataset, n_epochs=1000)
print(cmnl.weights)

We can compare the estimated coefficients and the negative log-likelihood obtained in \ref{} and \ref{torch-choice}, and it is similar !

In [ ]:
import tensorflow as tf

# Here are the values obtained in the references:
gt_weights = [
    tf.constant([[-0.0890796, -0.0279925, -0.038146]]),
    tf.constant([[-0.0333421]]),
    tf.constant([[0.0925304]]),
    tf.constant([[-0.0430032]]),
    tf.constant([[0.0595089, -0.00678188, -0.00645982, -0.00145029]]),
    tf.constant([[0.697311, 1.8437, 3.27381]]),
]
gt_model = ConditionalMNL(parameters=params, lr=0.01)
gt_model.fit(dataset, n_epochs=1, batch_size=-1)

# Here we estimate the negative log-likelihood with these coefficients (also, we obtain same value as in those papers):
gt_model.weights = gt_weights
gt_model.evaluate(dataset) * len(dataset)

In order to estimate the utilities, use the .predict_utility() method. In order to estimate the probabilities, use the .compute_probabilities() method.


In [ ]:
# print("Utilities of each item for the first 5 sessions:", cmnl.predict_utility(dataset)[:5])
print("Purchase probability of each item for the first 5 sessions:", cmnl.predict_probas(dataset)[:5])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history)

For very large datasets that do not fit entirely in the memory, the LBFGS method might not be the best choice. Here we can use the power of the Tensorflow library to use stochastic gradient descent optimizers.

In this case, it is possible to obtain the same coefficients estimation, also it is a little tricky to get it quickly. We need to adjust the learning rate over time for the optimization not to be too slow.

In [ ]:
cmnl = ConditionalMNL(parameters=params, optimizer="Adam")
history = cmnl.fit(dataset, n_epochs=2000, batch_size=-1)
cmnl.optimizer.lr = cmnl.optimizer.lr / 5
history2 = cmnl.fit(dataset, n_epochs=4000, batch_size=-1)
cmnl.optimizer.lr = cmnl.optimizer.lr  / 10
history3 = cmnl.fit(dataset, n_epochs=20000, batch_size=-1)

In [ ]:
cmnl.weights

In [ ]:
cmnl.evaluate(dataset)